In [ ]:
# Import required libraries and dependencies
import pandas as pd
import hvplot.pandas
from sklearn.cluster import KMeans
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler

In [ ]:
# Load the data into a Pandas DataFrame
df_market_data = pd.read_csv(
    "Resources/crypto_market_data.csv",
    index_col="coin_id")

# Display sample data
df_market_data.head(10)

In [ ]:
# Generate summary statistics
df_market_data.describe()

In [ ]:
# Plot your data to see what's in your DataFrame
df_market_data.hvplot.line(
    width=800,
    height=400,
    rot=90
)

---

### Prepare the Data

In [ ]:
# Use the `StandardScaler()` module from scikit-learn to normalize the data from the CSV file
df_market_data_scaled = StandardScaler().fit_transform(df_market_data)

df_market_data_scaled[:5]

In [ ]:
# Create a DataFrame with the scaled data
df_market_data_scaled = pd.DataFrame(df_market_data_scaled, columns= list(df_market_data.columns))

# Copy the crypto names from the original data

# Set the coinid column as index
df_market_data_scaled.index = df_market_data.index

# Display sample data
df_market_data_scaled

---

### Find the Best Value for k Using the Original Data.

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_scaled`
# 3. Append the model.inertia_ to the inertia list

for i in k:
    k_model=KMeans(n_clusters=i, random_state=1)
    k_model.fit(df_market_data_scaled)
    inertia.append(k_model.inertia_)


In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data={"k": k, "inertia": inertia}

# Create a DataFrame with the data to plot the Elbow curve
df_elbow = pd.DataFrame(elbow_data)
df_elbow

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_line_chart = df_elbow.hvplot.line(
    x="k",
    y="inertia",
    title="Elbow Curve",
    xticks=k)
elbow_line_chart

#### Answer the following question: 

**Question:** What is the best value for `k`?

**Answer:** 

---

### Cluster Cryptocurrencies with K-means Using the Original Data

In [ ]:
# Initialize the K-Means model using the best value for k
model=KMeans(n_clusters=3)

In [ ]:
# Fit the K-Means model using the scaled data
model.fit(df_market_data_scaled)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the scaled data
clusters = model.predict(df_market_data_scaled)

# Print the resulting array of cluster values.
clusters

In [ ]:
# Create a copy of the DataFrame
df_market_data_scaled_copy = df_market_data_scaled.copy()
df_market_data_scaled_copy

In [ ]:
# Add a new column to the DataFrame with the predicted clusters
df_market_data_scaled_copy["predicted_clusters"] = clusters

# Display sample data
clusters

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="price_change_percentage_24h"` and `y="price_change_percentage_7d"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.

scaled_clusters = df_market_data_scaled_copy.hvplot.scatter(
    x="price_change_percentage_24h",
    y="price_change_percentage_7d",
    by="predicted_clusters",
    hover_cols='coin_id'
)

---

### Optimize Clusters with Principal Component Analysis.

In [ ]:
# Create a PCA model instance and set `n_components=3`.
pca = PCA(n_components=3)

In [ ]:
# Use the PCA model with `fit_transform` to reduce to 
# three principal components.
model_fit_trans = pca.fit_transform(df_market_data_scaled)
# View the first five rows of the DataFrame. 
model_fit_trans[:5]

In [ ]:
# Retrieve the explained variance to determine how much information 
# can be attributed to each principal component.
pca.explained_variance_ratio_

#### Answer the following question: 

**Question:** What is the total explained variance of the three principal components?

**Answer:** 

In [ ]:
# Create a new DataFrame with the PCA data.
# Creating a DataFrame with the PCA data
pca_data_df = pd.DataFrame(model_fit_trans, columns=["PC1", "PC2", "PC3"])


# Copy the crypto names from the original data
pca_data_df["coin_id"] = df_market_data.index

# Set the coinid column as index
pca_data_df = pca_data_df.set_index("coin_id")

# Display sample data
pca_data_df

---

### Find the Best Value for k Using the PCA Data

In [ ]:
# Create a list with the number of k-values from 1 to 11
k = list(range(1,11))

In [ ]:
# Create an empty list to store the inertia values
inertia = []

# Create a for loop to compute the inertia with each possible value of k
# Inside the loop:
# 1. Create a KMeans model using the loop counter for the n_clusters
# 2. Fit the model to the data using `df_market_data_pca`
# 3. Append the model.inertia_ to the inertia list
for i in k:
    model = KMeans(n_clusters=i, random_state=0)
    model.fit(pca_data_df)
    inertia.append(model.inertia_)

In [ ]:
# Create a dictionary with the data to plot the Elbow curve
elbow_data_pca = {"k": k, "inertia": inertia}
# Create a DataFrame with the data to plot the Elbow curve
df_elbow_pca = pd.DataFrame(elbow_data_pca)

In [ ]:
# Plot a line chart with all the inertia values computed with 
# the different values of k to visually identify the optimal value for k.
elbow_plot_pca = df_elbow_pca.hvplot.line(
    x="k",
    y="inertia",
    title = "Elbow Curve",
    xticks=k)
elbow_plot_pca

#### Answer the following questions: 

* **Question:** What is the best value for `k` when using the PCA data?

  * **Answer:**


* **Question:** Does it differ from the best k value found using the original data?

  * **Answer:** 

### Cluster Cryptocurrencies with K-means Using the PCA Data

In [ ]:
# Initialize the K-Means model using the best value for k
model = KMeans(n_clusters=4)

In [ ]:
# Fit the K-Means model using the PCA data
model.fit(pca_data_df)

In [ ]:
# Predict the clusters to group the cryptocurrencies using the PCA data
pca_prediction = model.predict(pca_data_df)

# Print the resulting array of cluster values.
pca_prediction

In [ ]:
# Create a copy of the DataFrame with the PCA data
pca_data_df_copy = pca_data_df.copy()

# Add a new column to the DataFrame with the predicted clusters
pca_data_df_copy["predicted_clusters"] = pca_prediction

# Display sample data
pca_data_df_copy

In [ ]:
# Create a scatter plot using hvPlot by setting 
# `x="PC1"` and `y="PC2"`. 
# Color the graph points with the labels found using K-Means and 
# add the crypto name in the `hover_cols` parameter to identify 
# the cryptocurrency represented by each data point.
pca_clusters = pca_data_df_copy.hvplot.scatter(
    x="PC1",
    y="PC2",
    by = 'cluster_pca',
    hover_cols='coin_id'
)

#df_market_data_scaled_copy.hvplot.scatter(
 #   x="price_change_percentage_24h",
 #   y="price_change_percentage_7d",
 #   by="predicted_clusters",
 #   hover_cols='coin_id'
#) i tried comparing my work of these two and cannot find my error

### Visualize and Compare the Results

In this section, you will visually analyze the cluster analysis results by contrasting the outcome with and without using the optimization techniques.

In [ ]:
# Composite plot to contrast the Elbow curves
elbow_line_chart + elbow_plot_pca

In [ ]:
# Composite plot to contrast the clusters
scaled_clusters + pca_clusters

#### Answer the following question: 

  * **Question:** After visually analyzing the cluster analysis results, what is the impact of using fewer features to cluster the data using K-Means?

  * **Answer:** It seems that the clusters become more concentrated and separate in the PCA data.